In [8]:
import numpy as np

sents = []
iob_tags = []
text = ""
filename = './re3d/train.txt'
write_filename = "./re3d/sentences.txt"

with open(filename, 'r') as file:
    lst = []
    while True:
        word_lst : file.readline().lower().split("\t")
        word = word_lst[0]
        if not word:
            break
        text += " " + word

        if word == '\n':
            sents.append(lst)
            lst = []
        else:
            lst.append(word)
            iob_tags.append(word_lst[-1].split('\n')[0])

    sents.append(lst)

with open(write_filename, 'w') as f:
    f.write(text)
    f.close()

### Distinct Types of IOB Tokens

In [9]:
dist = set()
for tok in iob_tags:
    dist.add(tok)

print(dist)

{'b-organisation', 'b-quantity', 'b-weapon', 'i-documentreference', 'i-person', 'b-nationality', 'b-money', 'i-money', 'i-weapon', 'i-organisation', 'b-person', 'b-documentreference', 'i-quantity', 'i-militaryplatform', 'i-temporal', 'i-nationality', 'i-location', 'o', 'b-location', 'b-militaryplatform', 'b-temporal'}


In [10]:
# Here we are checking and ensuring that both number of tokens in sentences and number of tags in iob_tags are same
print('Number of iob tags are: ',len(iob_tags))

l=0
for i in range(len(sents)):
    l+=len(sents[i])

print('Number of tokens are: ',l)

Number of iob tags are:  20030
Number of tokens are:  20030


In [4]:
# converting iob tags to numerical tags
o = 0
b_org = 1
i_org = 2
b_quan = 3
i_quan = 4
b_weapon = 5
i_weapon = 6
b_doc = 7
i_doc = 8
b_person = 9
i_person = 10
b_nationality = 11
i_nationality = 12
b_money = 13
i_money = 14
b_temporal = 15
i_temporal = 16
b_loc = 17
i_loc = 18
b_milplat = 19
i_milplat = 20

iob_dict = {
    "o" : 0,
    "b_org" : 1,
    "i_org" : 2,
    "b_quan" : 3,
    "i_quan" : 4,
    "b_weapon" : 5,
    "i_weapon" : 6,
    "b_doc" : 7,
    "i_doc" : 8,
    "b_person" : 9,
    "i_person" : 10,
    "b_nationality" : 11,
    "i_nationality" : 12,
    "b_money" : 13,
    "i_money" : 14,
    "b_temporal" : 15,
    "i_temporal" : 16,
    "b_loc" : 17,
    "i_loc" : 18,
    "b_milplat" : 19,
    "i_milplat" : 20,
}
num_iob_tags = []
for tok in iob_tags:
    num_iob_tags.append(iob_dict[tok])

num_iob_tags = np.array(num_iob_tags)

In [5]:
from scipy.sparse import dok_matrix, vstack, csr_matrix
import random
from snorkel.labeling import PandasLFApplier, LFApplier, LFAnalysis, labeling_function


# helper functions
def dict_match(sentence, dictionary, max_ngrams=4):
   m = {}
   for i in range(len(sentence)):
       for j in range(i+1, min(len(sentence), i + max_ngrams) + 1):
           term = ' '.join(sentence[i:j])
           if term in dictionary:
               m.update({idx:1 for idx in range(i,j+1)})
   return m
           
         
def create_token_L_mat(Xs, Ls, num_lfs):
   """
   Create token-level LF matrix from LFs indexed by sentence
   """
   Yws = []
   for sent_i in range(len(Xs)):
       ys = dok_matrix((len(Xs[sent_i]), num_lfs))
       for lf_i in range(num_lfs):
           for word_i,y in Ls[sent_i][lf_i].items():
               ys[word_i, lf_i] = y
       Yws.append(ys)
   return csr_matrix(vstack(Yws))


  
# labeling functions
def LF_is_org(s):
   dict = {}
   if s[0]== 'the' and len(s)>2:
      dict[1] = b_org

   return dict

def weaponLF(s):
    label_dict = dict()
    # Returns a label of cuisine if the the phrase contains any of the words mentioned below
    weapon_words_list = ['weapons',
                            'strikes',
                            'chemical',
                            'rocket',
                            'artillery',
                            'munitions',
                            'strike',
                            'VBIED',
                            'gun',
                            'mortar',
                            'propelled',
                            'heavy',
                            'machine',
                            'VBIEDs',
                            'bombs',
                            'grenade',
                            'missile',
                            'air',
                            'bomb',
                            'barrel',
                            'launcher',
                            'front',
                            'end',
                            'weapon',
                            'systems',
                            'IED'
                        ]
    new_weapon_list = []
    for word in weapon_words_list:
        new_weapon_list.append(word.lower())
    tokens = s.lower()
    for i, word in enumerate(tokens):
        if word in new_weapon_list:
            if i > 0 and label_dict[i-1] == b_weapon:
                label_dict[i] = i_weapon
            else:
                label_dict[i] = b_weapon
    return label_dict


def nationalityLF(s):
    label_dict = dict()
    # Returns a label of cuisine if the the phrase contains any of the words mentioned below
    nationality_words_list =['Syrian',
                        'Iraqi',
                        'Kurdish',
                        'Yazidi',
                        'Arabic',
                        'Australian',
                        'Shia',
                        'Arab',
                        'Islam',
                        'Shiite',
                        'German',
                        'Jordan',
                        'Israeli',
                        'Muslims',
                        'Yazidis',
                        'Kurmanji',
                        'Northern'
                        ]
    new_nationality_list = []
    for word in nationality_words_list:
        new_nationality_list.append(word.lower())
    tokens = s.lower()
    for i, word in enumerate(tokens):
        if word in new_nationality_list:
            if i > 0 and label_dict[i-1] == b_nationality:
                label_dict[i] = i_nationality
            else:
                label_dict[i] = b_nationality
    return label_dict


def LF_is_loc(s):
   dict = {}
   # for i in range(len(s)):
   #    p = random.random()
   #    if p>=0.5:
   #       dict[i] = I_ORG
   return dict



lfs = [
   LF_is_org,
   LF_is_per,
   LF_is_loc
]

# apply labeling functions and transform label matrix 
L = [[lf(s) for lf in lfs] for s in sents] 

L = create_token_L_mat(sents, L, len(lfs))  
L = np.asarray(L.astype(np.int8).todense())

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=4, verbose=True)
label_model.fit(L_train=L, n_epochs=500, log_freq=100, seed=123)

label_model.predict(L)


In [ ]:
# finding accuracy
label_model_acc = label_model.score(L=L, Y=num_iob_tags, tie_break_policy="random")["accuracy"]
print(label_model_acc)